### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
import pandas as pd
import numpy as np

# set up a student and a schools df
schools_path = "Resources/schools_complete.csv"
students_path = "Resources/students_complete.csv"
schools_df = pd.read_csv(schools_path)
students_df = pd.read_csv(students_path)
school_student_combo = students_df.merge(schools_df, how='left', on='school_name')
school_student_combo.set_index('Student ID', inplace=True)

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)
* Create a dataframe to hold the above results
* Optional: give the displayed data cleaner formatting

In [33]:
# start with number of students
num_students = students_df.count()['student_name']
num_schools = schools_df.count()['school_name']
budget = schools_df.sum()['budget']
math_column = students_df['math_score']
reading_column = students_df['reading_score']
ave_math = round(math_column.mean(), 2)
ave_reading = round(reading_column.mean(), 2)
num_passing_math = math_column[math_column>69].count()
pct_passing_math = round(num_passing_math / num_students * 100, 2)
num_passing_reading = reading_column[reading_column>69].count()
pct_passing_reading = round(num_passing_reading / num_students * 100, 2)

math_and_reading = students_df[['reading_score', 'math_score']]
passing_all = math_and_reading[
    (math_and_reading['reading_score']>69) 
    & (math_and_reading['math_score']>69)
].shape[0]
pct_passing_all = round(passing_all / num_students * 100, 2)

In [ ]:
# Build summary df
formatted_num_students = np.array2string(num_students, formatter={'int_kind': lambda x: '{:,}'.format(x)})
formatted_budget = np.array2string(budget, formatter={'int_kind': lambda x: '${:,.2f}'.format(x)})
district_summary_df = pd.DataFrame({
    'Number of Schools': num_schools,
    'Number of Students': formatted_num_students,
    'Total Budget': formatted_budget,
    'Average Math Score': ave_math,
    'Average Reading Score': ave_reading,
    '% Passing Math': pct_passing_math,
    '% Passing Reading': pct_passing_reading,
    '% Overall Passing': pct_passing_all
}, index=[0])
district_summary_df.head()

## School Summary
  * Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.) 
  * Create a dataframe to hold the above results

In [ ]:
school_students_df = school_student_combo.groupby('school_name').count()['student_name']
school_students_df.rename('Total Students', inplace=True)
school_budget_df = schools_df.set_index('school_name')['budget']
school_budget_df.rename('Total School Budget', inplace=True)
school_type_df = schools_df.set_index('school_name')['type']
school_type_df.rename('School Type', inplace=True)
school_budget_norm_df = round(school_budget_df / school_students_df, 2)
school_budget_norm_df.rename('Per Student Budget', inplace=True)
school_ave_math_df = round(school_student_combo.groupby('school_name').mean()['math_score'],2)
school_ave_math_df.rename('Average Math Score', inplace=True)
school_ave_reading_df = round(school_student_combo.groupby('school_name').mean()['reading_score'],2)
school_ave_reading_df.rename('Average Reading Score', inplace=True)
school_num_passing_math = school_student_combo[school_student_combo['math_score']>69].groupby('school_name').count()['math_score']
school_pct_passing_math = round(school_num_passing_math / school_students_df * 100, 2)
school_pct_passing_math.rename('% Passing Math', inplace=True)
school_num_passing_reading = school_student_combo[school_student_combo['reading_score']>69].groupby('school_name').count()['reading_score']
school_pct_passing_reading = round(school_num_passing_reading / school_students_df * 100, 2)
school_pct_passing_reading.rename('% Passing Reading', inplace=True)

school_passing_all = school_student_combo[
    (school_student_combo['math_score']>69)
    & (school_student_combo['reading_score']>69)
].groupby('school_name').count()['student_name']
school_pct_passing_all = round(school_passing_all / school_students_df * 100, 2)
school_pct_passing_all.rename('% Overall Passing', inplace=True)

# create summary DataFrame
school_summary_df = pd.DataFrame({
    school_type_df.name: school_type_df,
    school_students_df.name: school_students_df,
    school_budget_df.name: school_budget_df.map('${:,.2f}'.format),
    school_budget_norm_df.name: school_budget_norm_df.map('${:,.2f}'.format),
    school_ave_math_df.name: school_ave_math_df,
    school_ave_reading_df.name: school_ave_reading_df,
    school_pct_passing_math.name: school_pct_passing_math,
    school_pct_passing_reading.name: school_pct_passing_reading,
    school_pct_passing_all.name: school_pct_passing_all
})
school_summary_df.index.name = 'School Name'
school_summary_df

## Top Performing Schools, by % Overall Passing
* Sort and display the top five performing schools by % overall passing.

In [ ]:
top_schools_df = school_summary_df.sort_values('% Overall Passing', ascending=False)
top_schools_df.head(5)

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [ ]:
top_schools_df.sort_values('% Overall Passing').head(5)

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
